# De la description textuelle à l'image

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Imports et telechargements

In [ ]:
import json
import pandas as pd

## Récup data

In [ ]:
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans0.txt', encoding='utf-16')
text0 = file.read()
d = json.loads(text0)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans1.txt', encoding='utf-16')
text1 = file.read()
d1 = json.loads(text1)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans2.txt', encoding='utf-16')
text2 = file.read()
d2 = json.loads(text2)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans_rest.txt', encoding='utf-16')
text3 = file.read()
d3 = json.loads(text3)
dictionnary = d | d1
dictionnary = dictionnary | d2
dictionnary = dictionnary | d3
len(dictionnary)
descr = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/Transcription_wav.csv')
name_video = descr['name_vid']
contenue = descr['contenue']
dic = {}
vid_name_set = set()
for vid_name, cont in zip(list(name_video), list(contenue)):
  if not 'artificial intelligence' in cont.lower():
    if not vid_name in dic.keys():
      dic[vid_name] = cont
    else:
      dic[vid_name] += cont
  else:
    vid_name_set.add(vid_name)
    print(vid_name)

def function_reu(x):
  try:
    return dic[x]
  except:
    return ''

def function_test(x):
  try:
    return dictionnary[(x+ '.mp3')]
  except:
    return 'do not have'

descr['descr'] = descr['name_vid'].apply(lambda x : function_reu(x))
descr = descr.drop_duplicates(subset='name_vid')
dt = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/LVT.csv')
dt['name'] = dt['video'].apply(lambda x : x[5:-4])
dt['transcription'] = dt['name'].apply(lambda x : function_test(x))
dt = dt.drop('video', axis=1)
dt = dt[dt.transcription != 'do not have']
intersection = pd.merge(dt, descr, left_on='name', right_on='name_vid')
intersection = intersection.drop('label', axis = 1)
intersection = intersection.drop('Unnamed: 0', axis = 1)
intersection = intersection.drop('name_audio', axis = 1)
intersection = intersection.drop('id_audio', axis = 1)
intersection = intersection.drop('name_vid', axis = 1)
intersection = intersection.drop('contenue', axis = 1)
intersection['total'] = intersection['text'] + intersection['descr'] + intersection['transcription']
print(len(intersection), len(dt), len(descr))

-eOrNdBpGMv8
1551 2085 1878


In [ ]:
dt

,duree,label,text,label_encoded,name,transcription
0,77.60,Entertainment,"cut to Black, Carl a man in a long tan Overcoa...",0,-dXyrxjIcWdw,This has to be the place. No cameras. That's ...
1,318.72,Music,the camera from the helicopter captures the ru...,1,-g0FMrjsYAzc,The beginning part of the season kind of suck...
2,163.42,Education,a blindfold adult wearing green sits next to a...,2,-h7HGWRTGqyg,I'm gonna staple some more. There we go. Good...
3,125.02,Film & Animation,"a man doing a trick in the air, Robert Downey ...",3,-eOrNdBpGMv8,You were made to be ruled. In the end... It w...
4,171.74,Film & Animation,crisp white text Upon A moonlit background Ree...,3,-dRaKMNYG3-I,You I I'm sorry. You I I Oh You You
...,...,...,...,...,...,...
2094,190.03,Entertainment,the supermarket spray logo appears on the scre...,0,--UggNH1YeRY,You're watching the Game Show Network. I watc...
2095,55.50,Education,Barbara grabs a red cup from her right and put...,2,--kNSaU8L3mI,"For another child, even something like notici..."
2096,158.59,Entertainment,"dad Smiles his lips move when he speaks, the m...",0,-0-NBRA1aSXk,Music Dad Dad Dad. Dad. Dad. Dad. Dad has voi...
2097,108.62,People & Blogs,add 3 cups of flour a teaspoon of salt and a t...,4,--bCmuE3KxWI,Music you you you you


## Récup model

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.5 MB/s eta 0:00:00


In [ ]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

'\nurl = "http://images.cocodataset.org/val2017/000000039769.jpg"\nimage = Image.open(requests.get(url, stream=True).raw)\n\ninputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)\n\noutputs = model(**inputs)\nlogits_per_image = outputs.logits_per_image  # this is the image-text similarity score\nprobs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities\n'

## Récup images de la vidéo

In [ ]:
!pip install av

In [ ]:
def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [ ]:
import av
import numpy as np
index_frames_matrix = np.load('/content/gdrive/MyDrive/Projet_Multimedia/download/KeyImages/FinalMatrixFramesIndices.npy')
vid_folder_path = "/content/gdrive/MyDrive/Projet_Multimedia/videos/Video_with_label/"
container = av.open('/content/gdrive/MyDrive/Projet_Multimedia/videos/Video_with_label/' + 'video-dRaKMNYG3-I.mp4')
video = read_video_pyav(container=container, indices=index_frames_matrix[2094])

## Appliquer modèle permettant de résumer les textes

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def sumarize_txt (texte) :
  txt = summarizer(texte, max_length=15, min_length=10, do_sample=False)[0]['summary_text']
dt['text_summarized'] = dt.text.apply(lambda x : sumarize_txt(x))

In [ ]:
dt.to_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CVS/data_text_sum.csv')
# enreg le resultat sur drive

## Lien image-texte

On regarde pour chaque video quel est le texte le plus probable parmis tous ceux que l'on a mais la RAM ne supporte plus alors on est un peu bloqués


In [ ]:
# for image in video :
from tqdm import tqdm
arg_max = []
for image in tqdm(video) :
  textes = list(dt['text_summarized'])
  inputs = processor(text=textes, images=image, return_tensors="pt", padding=True)
  outputs = model(**inputs)
  logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
  probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
  arg_max.append(np.argmax(probs.detach().numpy()))

arg_max

100%|██████████| 32/32 [01:51<00:00,  3.48s/it]
